<a href="https://colab.research.google.com/github/javgonzalezvi/MetNumUN2022I/blob/master/Lab9/week_4_iterative_linalg_Jacobi_Seidel_minres_Group8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [40]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [41]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [42]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [43]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [44]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [45]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [46]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [47]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [48]:
def jacobi(A, b, eps, num_iter):
    
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b
    
    x = np.ones(n)
    for _ in range(n_iter):
        x = BB @ x + c
    return x

In [49]:
eps = 1e-12
num_iter = 50 

x = jacobi(A, b, eps, num_iter)
#print(x)

np.testing.assert_allclose(A @ x, b)
np.testing.assert_allclose(x, xx)

In [50]:
eps = 1e-12
num_iter = 50 

for k in range(1, 25):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1)) @ (-A1.copy() + np.diag(np.diag(A1)))),
          np.linalg.norm(jacobi(A1, b, eps, num_iter) - np.linalg.solve(A1, b)))

0.38959181027260875 2.005305120107153e-17
0.4185783948614869 1.6502682545529e-17
0.4522284025473819 3.622208959480119e-17
0.4917667095178099 2.72080745804227e-17
0.5388887887486234 2.660694867584809e-17
0.5960110344093966 1.0255972154762448e-15
0.6667001660296402 2.770910834768342e-13
0.7564517359241753 1.4990529163071287e-10
0.8742017351588476 2.0161596093475135e-07
1.0355299928250665 0.0009191717405677889
1.2702850939751231 23.48163367953916
1.6439565658213244 8260242.793633645
2.334809111760855 261149802433164.78
4.080768845910033 1.3716691464130671e+26
30.715327603064885 1.739863582849129e+61
25.668433387443834 7.8814393778775e+55
3.8385533278362765 1.1479079818548211e+25
2.2533561772001827 52372118749887.13
1.6031859449057577 1627830.563305912
1.24581575811634 46.233686175462694
1.019215329207032 0.0008730836032709015
0.8625478003698426 1.6544090716892408e-07
0.7477110355740726 1.2057564079371141e-10
0.6599017255790174 2.2409618549351615e-13


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [51]:
def seidel(A, b, eps, num_iter):
  
    x = np.ones(b.shape[0])
    
    for _ in range(num_iter):
        for k in range(b.shape[0]):
              x[k] = (b[k]-np.dot(A[k][:k], x[:k])- np.dot(A[k][k+1:], x[k+1:]))/A[k,k]
    return x

In [52]:
eps = 1e-12
num_iter = 50 

x = seidel(A,b,eps, num_iter)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [53]:
eps = 1e-12
num_iter = 50 

for k in range(1, 25):

    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1)) @ (-A1.copy() + np.diag(np.diag(A1)))),
          np.linalg.norm(seidel(A1, b, eps, num_iter) - np.linalg.solve(A1, b)))

0.38959181027260875 1.5540063044689707e-17
0.4185783948614869 2.303412968156487e-17
0.4522284025473819 3.3107778123195866e-17
0.4917667095178099 1.717836144195444e-17
0.5388887887486234 1.8703665918870363e-17
0.5960110344093966 2.6208106743381504e-17
0.6667001660296402 3.222105849667643e-17
0.7564517359241753 3.894444544739273e-17
0.8742017351588476 4.3610271956070115e-17
1.0355299928250665 7.521581756278068e-17
1.2702850939751231 6.691626947686432e-17
1.6439565658213244 1.0829177162645093e-16
2.334809111760855 1.1775693440128312e-16
4.080768845910033 4.10633611443092e-09
30.715327603064885 2.224327617064848e+114
25.668433387443834 inf
3.8385533278362765 1.5806625506791825e+58
2.2533561772001827 9.131420895207482e+28
1.6031859449057577 2821070777462.6245
1.24581575811634 160.48281049788977
1.019215329207032 2.8667224812005137e-07
0.8625478003698426 4.767678402928541e-14
0.7477110355740726 4.8074067159589095e-17
0.6599017255790174 6.938893903907228e-17


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [54]:
def minimum_res_scheme(A, b, eps, num_iter):

    x = np.ones(b.shape[0])
    
    for _ in range(n_iter):
        r = A @ x - b
        k = (r @ A @ r)/(np.linalg.norm(A @ r)**2)
        x = x - k*r
        
    return x


In [55]:
eps = 1e-12
num_iter = 50 

x = minimum_res_scheme(A,b, eps, num_iter)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [56]:
for k in range(1, 25):

    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1)) @ (-A1.copy() + np.diag(np.diag(A1)))),
          np.linalg.norm(minimum_res_scheme(A1, b, eps, num_iter) - np.linalg.solve(A1, b)))

0.38959181027260875 2.224174099750403e-17
0.4185783948614869 2.32747678105055e-17
0.4522284025473819 3.867305227724097e-17
0.4917667095178099 2.823925224926892e-17
0.5388887887486234 1.717288626685379e-17
0.5960110344093966 2.3338317471424865e-17
0.6667001660296402 3.309641454705087e-17
0.7564517359241753 nan
0.8742017351588476 4.718958039861916e-17
1.0355299928250665 7.11870997173569e-17
1.2702850939751231 6.834017936133213e-17
1.6439565658213244 1.0367784962136927e-16
2.334809111760855 1.0822660244255038e-13
4.080768845910033 0.00021717913683793173
30.715327603064885 2.0818042350846104
25.668433387443834 1.3617048070015483
3.8385533278362765 0.2394689869892452
2.2533561772001827 0.3433002155869048
1.6031859449057577 1.0711140128530185
1.24581575811634 13.13392245452631
1.019215329207032 6.777506492437432e-09
0.8625478003698426 2.475075345743422e-15
0.7477110355740726 6.359601310784502e-17
0.6599017255790174 1.0838898772828417e-16


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
